In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import nltk 
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [49]:
#import business artical
import os
text = []
for root, dirs, files in os.walk("business"):
   for file in files:
       if file.endswith(".txt"):
            filename = os.path.join(root, file) 
            with open(filename, 'r', encoding="utf8", errors='ignore') as infile:
                intext = ''
                for line in infile:
                    intext = intext + ' ' + line.replace("\n", '')
                text.append(intext)
            


In [50]:
df = pd.DataFrame(text, columns=['text'])

In [51]:
texts = pd.Series(text)

In [52]:
texts[5]

' Japan narrowly escapes recession  Japan\'s economy teetered on the brink of a technical recession in the three months to September, figures show.  Revised figures indicated growth of just 0.1% - and a similar-sized contraction in the previous quarter. On an annual basis, the data suggests annual growth of just 0.2%, suggesting a much more hesitant recovery than had previously been thought. A common technical definition of a recession is two successive quarters of negative growth.  The government was keen to play down the worrying implications of the data. "I maintain the view that Japan\'s economy remains in a minor adjustment phase in an upward climb, and we will monitor developments carefully," said economy minister Heizo Takenaka. But in the face of the strengthening yen making exports less competitive and indications of weakening economic conditions ahead, observers were less sanguine. "It\'s painting a picture of a recovery... much patchier than previously thought," said Paul Sh

In [53]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\e175932\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
# collect counts:
probs = {} # key: (w(t-1), w(t+1), value: {w(t): count(w(t))})
for doc in texts:
    lines = doc.split("\n")
    for line in lines:
        tokens = word_tokenize(line)
        for i in range(len(tokens) - 2):
            t_0 = tokens[i]
            t_1 = tokens[i+1]
            t_2 = tokens[i +2]
            key = (t_0, t_2)
            if key not in probs:
                probs[key] = {}
            # add count for middle token
            if t_1 not in probs[key]:
                probs[key][t_1] = 1
            else: 
                probs[key][t_1] += 1

In [55]:
# normalize probabilities 
for key, d in probs.items():
    # d should represent a distribution
    total = sum(d.values())
    for k, v in d.items():
        d[k] = v/total


In [56]:
dict(list(probs.items())[:10]) #checking probs

{('Ad', 'boost'): {'sales': 1.0},
 ('sales', 'Time'): {'boost': 1.0},
 ('boost', 'Warner'): {'Time': 1.0},
 ('Time', 'profit'): {'Warner': 1.0},
 ('Warner', 'Quarterly'): {'profit': 1.0},
 ('profit', 'profits'): {'Quarterly': 1.0},
 ('Quarterly', 'at'): {'profits': 1.0},
 ('profits', 'US'): {'at': 1.0},
 ('at', 'media'): {'US': 1.0},
 ('US', 'giant'): {'media': 0.1,
  'telecoms': 0.1,
  'banking': 0.2,
  'foods': 0.1,
  'retail': 0.1,
  'oil': 0.2,
  'mortgage': 0.1,
  'agrochemical': 0.1}}

In [57]:
def spin_document(doc):
    #split the document into lines(paragrah)
    lines = doc.split("\n")
    output = []
    for line in lines:
        if line:
            new_line = spin_line(line)
        else:
            new_line = line
        output.append(new_line)
    return "\n".join(output)

In [58]:
detokenizer = TreebankWordDetokenizer()

In [59]:
#df['text'].iloc[0].split("\n")[2] #checking one example

In [60]:
#detokenizer.detokenize(word_tokenize(df['text'].iloc[0].split("\n")[2]))

In [61]:
def sample_word(d):
    p0 = np.random.random()
    cummulative = 0
    for t, p in d.items():
        cummulative += p 
        if p0 < cummulative:
            return t
    assert(False) # should never get there

In [62]:
from nltk.tokenize import  word_tokenize
def spin_line(line):
    tokens = word_tokenize(line)
    i = 0 
    output = [tokens[0]]
    while i < (len(tokens) - 2):
        t_0 = tokens[i]
        t_1 = tokens[i + 1]
        t_2 = tokens[i + 2]
        key = (t_0, t_2)
        p_dist = probs[key]
        if len(p_dist) > 1 and np.random.random() < 0.3:
            #let's replace the middle word
            middle = sample_word(p_dist)
            output.append(t_1)
            output.append("<" + middle + ">")
            output.append(t_2)
            
            #we wont replace the 3rd token since the middle 
            # token was dependent on it 
            # instead, skp ahead 2 steps 
            i += 2
        else:
            # we wont replace this middle word 
            output.append(t_1)
            i += 1
    # append the final token  - only if there is no replacement
    if i == len(tokens) - 2:
        output.append(tokens[-1])
    return detokenizer.detokenize(output)

In [63]:
i = np.random.choice(texts.shape[0])
doc = texts.iloc[i]
new_doc = spin_document(doc)

In [64]:
import textwrap
print(textwrap.fill(
    new_doc, replace_whitespace = False, fix_sentence_endings = True
))

'Golden economic period' to end Ten years of <.> "golden
<significant>" economic performance may come <choose> to an end
<improvement> in 2005 with growth slowing markedly, City <City>
consultancy Deloitte has warned . The UK economy could suffer a
backlash <quarter> from the slowdown <dip> in the housing market,
triggering <despite> a fall in consumer <consumer> spending and <and>
a rise in unemployment . Deloitte is forecasting economic growth of 2
<5.1>% this year <quarter>, below Chancellor Gordon Brown's forecast
<purchase> of 3% <%> to 3.5% . It <It> also believes <recommends> that
interest rates will fall to 4 <29>% by <of> the end of <of> the year
<increase>. In its quarterly <likely> economic review <growth>,
Deloitte <analysts> said the UK economy had enjoyed a "golden period"
during the past decade with unemployment falling to a <a> near 30 year
low and inflation at its lowest <highest> since the 1960s . But it
<it> warned that this growth had been achieved at <at> the expen